# importing necessary packages

In [11]:
import json
import random
import logging
import os
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
import spacy

# main_functions for model building

In [12]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()
        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]
                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [13]:
def train_spacy():
    TRAIN_DATA = convert_dataturks_to_spacy("/home/ajay/Downloads/new _resumes.json")
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(100):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update([text],[annotations],drop=0.2,sgd=optimizer,losses=losses)
        current_directory = "/home/ajay/spacy_models/"
        final_directory = os.path.join(current_directory,'spacy_updated_model'+ " " +str(losses))
        if not os.path.exists(final_directory):
            os.makedirs(final_directory)
            nlp.to_disk(final_directory)
            print("Saved model")

In [ ]:
train_spacy()

Statring iteration 0
Statring iteration 1
Statring iteration 2
Statring iteration 3
Statring iteration 4
Statring iteration 5
Statring iteration 6
Statring iteration 7
Statring iteration 8
Statring iteration 9
Statring iteration 10
Statring iteration 11
Statring iteration 12
Statring iteration 13
Statring iteration 14
Statring iteration 15
Statring iteration 16
Statring iteration 17
Statring iteration 18
Statring iteration 19
Statring iteration 20
Statring iteration 21
Statring iteration 22
Statring iteration 23
Statring iteration 24
Statring iteration 25
Statring iteration 26
Statring iteration 27
Statring iteration 28
Statring iteration 29


# loading model and testing with resume

In [ ]:
nlp2 = spacy.load("your_model_file")
examples = textract.process("resume path")
examples = examples.decode()
doc = nlp2(examples)
for ent in doc.ents:
    a = {ent.label_:ent.text}
    print(a)